# 02_ocr_testing.ipynb
## (Refactorizado) Paso 2: Extracción con API ADE

**Objetivo:** Probar el endpoint `v1/ade/parse` de Landing AI para extraer Markdown estructurado de:
1. Los **PDFs originales** (de `data/input/pdf`)
2. Las **imágenes de Excel** (de `data/images/excel_images`)

In [1]:
import sys
import logging
from pathlib import Path
import json
import os
import requests # Necesario para las llamadas a la API

# --- Celda de Configuración Estándar ---
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import src.utils.config 
from src.utils.config import (
    LANDING_AI_API_KEY, # Importamos la clave
    INPUT_PDF_DIR,      
    EXCEL_IMAGES_DIR,   
    EXTRACTED_TEXT_DIR  
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"Raíz del proyecto establecida en: {PROJECT_ROOT}")
print("✅ Entorno configurado. Logging listo.")

Raíz del proyecto establecida en: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent
✅ Entorno configurado. Logging listo.


## 1. Funciones de Prueba de API (Locales)

In [2]:
# --- Configuración de API Key ---
# Usamos la clave importada de config.py
MI_API_KEY = LANDING_AI_API_KEY
ADE_API_URL = "https://api.va.landing.ai/v1/ade/parse"

if not MI_API_KEY or "..." in MI_API_KEY:
    logger.error("¡ADVERTENCIA! Reemplaza '...' en tu .env con tu API Key de Landing AI.")
    # Levantamos un error para detener la ejecución si la clave falta
    raise ValueError("Falta LANDING_AI_API_KEY. Revisa tu archivo .env")
else:
    print("API Key de Landing AI cargada.")

def extract_ade_from_pdf(pdf_path: Path) -> dict:
    """Envía un PDF COMPLETO al endpoint ADE."""
    logger.info(f"Procesando PDF: {pdf_path.name} con ADE...")
    headers = {"Authorization": f"Bearer {MI_API_KEY}"}
    if not pdf_path.exists():
        return {"status": "error", "message": "Archivo no encontrado"}

    with open(pdf_path, 'rb') as f_pdf:
        files_payload = {'document': (pdf_path.name, f_pdf, 'application/pdf'), 'model': (None, 'dpt-2-latest')}
        try:
            response = requests.post(ADE_API_URL, headers=headers, files=files_payload, timeout=120)
            response.raise_for_status()
            return {"status": "success", "provider": "landing_ai_ade_pdf", "source_file": pdf_path.name, "text": response.json().get('markdown', '')}
        except requests.exceptions.RequestException as e:
            logger.error(f"Error de API (PDF) para {pdf_path.name}: {e}")
            return {"status": "error", "message": str(e), "source_file": pdf_path.name}

def extract_ade_from_image(image_path: Path) -> dict:
    """Envía una IMAGEN (PNG) al endpoint ADE."""
    logger.info(f"Procesando Imagen: {image_path.name} con ADE...")
    headers = {"Authorization": f"Bearer {MI_API_KEY}"}
    if not image_path.exists():
        return {"status": "error", "message": "Archivo no encontrado"}

    with open(image_path, 'rb') as f_img:
        files_payload = {'document': (image_path.name, f_img, 'image/png'), 'model': (None, 'dpt-2-latest')}
        try:
            response = requests.post(ADE_API_URL, headers=headers, files=files_payload, timeout=60)
            response.raise_for_status()
            return {"status": "success", "provider": "landing_ai_ade_image", "source_file": image_path.name, "text": response.json().get('markdown', '')}
        except requests.exceptions.RequestException as e:
            logger.error(f"Error de API (Imagen) para {image_path.name}: {e}")
            return {"status": "error", "message": str(e), "source_file": image_path.name}

print("Funciones 'extract_ade_from_pdf' y 'extract_ade_from_image' definidas para prueba.")

API Key de Landing AI cargada.
Funciones 'extract_ade_from_pdf' y 'extract_ade_from_image' definidas para prueba.


## 2. Ejecutar Extracción (Simulando Paso 2 del pipeline)

In [3]:
# Listar todos los archivos de entrada que vamos a procesar
pdf_files = list(INPUT_PDF_DIR.glob("*.pdf"))
excel_png_files = list(EXCEL_IMAGES_DIR.glob("*.png"))

print(f"PDFs originales encontrados: {len(pdf_files)}")
print(f"Imágenes PNG de Excel encontradas: {len(excel_png_files)}")

all_document_paths = pdf_files + excel_png_files
print(f"Total de documentos a procesar: {len(all_document_paths)}")

all_ade_results = []

PDFs originales encontrados: 1
Imágenes PNG de Excel encontradas: 8
Total de documentos a procesar: 9


In [8]:
print("="*60)
print("INICIANDO EXTRACCIÓN DE MARKDOWN (ADE)")
print("="*60)

for i, doc_path in enumerate(all_document_paths, 1):
    
    if doc_path.suffix.lower() == ".pdf":
        result = extract_ade_from_pdf(doc_path)
        prefix = "PDF"
    else:
        result = extract_ade_from_image(doc_path)
        prefix = "Imagen"
        
    all_ade_results.append(result)
    
    if result['status'] == 'success':
        print(f"  [{i}/{len(all_document_paths)}] ✅ Éxito ({prefix}): {doc_path.name}. Caracteres: {len(result['text'])}")
    else:
        print(f"  [{i}/{len(all_document_paths)}] ❌ Error ({prefix}): {doc_path.name}. Mensaje: {result['message']}")
print("="*60)

INFO:__main__:Procesando PDF: 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR.pdf con ADE...


INICIANDO EXTRACCIÓN DE MARKDOWN (ADE)


INFO:__main__:Procesando Imagen: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r0_c0.png con ADE...


  [1/9] ✅ Éxito (PDF): 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR.pdf. Caracteres: 2914


INFO:__main__:Procesando Imagen: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r0_c200.png con ADE...


...

  [6/9] ✅ Éxito (Imagen): 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r150_c0.png. Caracteres: 30211


INFO:__main__:Procesando Imagen: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c200.png con ADE...


  [7/9] ✅ Éxito (Imagen): 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c0.png. Caracteres: 49274


INFO:__main__:Procesando Imagen: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c250.png con ADE...


  [8/9] ✅ Éxito (Imagen): 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c200.png. Caracteres: 12245
  [9/9] ✅ Éxito (Imagen): 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c250.png. Caracteres: 12854


## 3. Guardar Resultados (Simulando Pipeline)

In [9]:
print(f"\nProcesamiento completado. Total de resultados: {len(all_ade_results)}")

# Guardamos los resultados en la carpeta que usará el siguiente paso (Parser)
success_count = 0
EXTRACTED_TEXT_DIR.mkdir(parents=True, exist_ok=True) # Asegurarse de que exista

for result in all_ade_results:
    if result['status'] == 'success':
        base_name = Path(result['source_file']).stem
        output_filename = f"{base_name}_text.json"
        output_path = EXTRACTED_TEXT_DIR / output_filename
        
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            success_count += 1
        except Exception as e:
            logger.error(f"Error al guardar {output_filename}: {e}")

print(f"Resultados exitosos guardados en {EXTRACTED_TEXT_DIR.relative_to(PROJECT_ROOT)}: {success_count}")


Procesamiento completado. Total de resultados: 9
Resultados exitosos guardados en data\processed\extracted_text: 9


## 4. Prueba de Parseo (Simulando Paso 3)

In [10]:
# Ahora, probemos el siguiente paso del pipeline: el StructureParser (PASO 4)
# Importamos las clases y funciones del Paso 4
from src.extractors.structure_parser import StructureParser, OperacionNave
from langchain_openai import ChatOpenAI
from src.utils.config import LLM_MODEL, LLM_TEMPERATURE, OPENAI_API_KEY

parser = None
if not OPENAI_API_KEY or "..." in OPENAI_API_KEY:
    print("⚠️ No se encontró OPENAI_API_KEY. Saltando prueba de parseo.")
    print("   Configúrala en tu .env si quieres probar el Paso 4.")
else:
    # Inicializar el LLM y el Parser (Markdown -> OperacionNave JSON)
    llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)
    parser = StructureParser(llm=llm, schema=OperacionNave)
    print(f"✅ Parser (Markdown -> OperacionNave JSON) inicializado con modelo: {LLM_MODEL}")


2025-11-08 15:17:07,382 - src.extractors.structure_parser - INFO - StructureParser inicializado con el esquema: OperacionNave


INFO:src.extractors.structure_parser:StructureParser inicializado con el esquema: OperacionNave


✅ Parser (Markdown -> OperacionNave JSON) inicializado con modelo: gpt-5-nano


In [13]:
# Ejecutar el Parseo sobre el primer resultado exitoso (con nuevos campos de OperacionNave)
from pathlib import Path
import json
from src.utils.config import EXTRACTED_TEXT_DIR, EXTRACTED_TABLES_DIR

if parser and all_ade_results:
    # Buscar el primer resultado exitoso para probar
    first_success = next((r for r in all_ade_results if r.get('status') == 'success'), None)
    
    if first_success:
        markdown_text = first_success.get('text')
        source_name = first_success.get('source_file', 'documento')
        stem = Path(source_name).stem

        print(f"\n--- Parseando: {stem}_text.json ---")
        
        try:
            structured_data = parser.parse_document(markdown_text)
            
            # Guardar el resultado (simulando el pipeline real) en extracted_tables
            output_filename = f"{stem}_structure.json"
            output_path = EXTRACTED_TABLES_DIR / output_filename
            output_path.parent.mkdir(parents=True, exist_ok=True)
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(structured_data, f, ensure_ascii=False, indent=2)
            
            print(f"✅ ¡Parseo Exitoso! -> {output_path.name}")
            print("-" * 50)

            # --- NUEVOS CAMPOS (OperacionNave) ---
            print(f"Recalada: {structured_data.get('recalada')}")
            print(f"Nave: {structured_data.get('nave_nombre')}")
            print(f"Inicio de operación: {structured_data.get('fecha_inicio_operacion')}")
            print(f"Fin de operación: {structured_data.get('fecha_fin_operacion')}")
            print("-" * 50)

            # Bodegas con tonelaje
            bodegas = structured_data.get('bodegas', []) or []
            if bodegas:
                print("Bodegas con tonelaje:")
                for b in bodegas:
                    bodega = b.get('bodega', '-')
                    tonelaje = b.get('tonelaje', '-')
                    lotes = ", ".join(b.get('lotes', []) or []) or '-'
                    print(f"  • {bodega}: {tonelaje} t | Lotes: {lotes}")
            else:
                print("Bodegas con tonelaje: -")
            print("-" * 50)

            # Clientes atendidos
            clientes = structured_data.get('clientes', []) or []
            print(f"Clientes atendidos ({len(clientes)}): {', '.join(clientes) if clientes else '-'}")
            print("-" * 50)

            # Lotes ↔ Facturación
            lotes_fact = structured_data.get('lotes_facturacion', []) or []
            if lotes_fact:
                print("Lotes ↔ Facturación:")
                for lf in lotes_fact:
                    lote = lf.get('lote', '-')
                    cliente = lf.get('cliente', '-')
                    bodega = lf.get('bodega', '-')
                    tonelaje = lf.get('tonelaje')
                    tonelaje_str = f"{tonelaje} t" if tonelaje is not None else "-"
                    cods = ", ".join(lf.get('codigos_facturacion', []) or []) or "-"
                    print(f"  • Lote {lote} | Cliente: {cliente} | Bodega: {bodega} | Tonelaje: {tonelaje_str} | Facturas: {cods}")
            else:
                print("Lotes ↔ Facturación: -")
            print("-" * 50)

            # Vista inteligente (si existe)
            vista = structured_data.get('vista_inteligente')
            if vista:
                print("Vista inteligente:")
                print(vista)
                print("-" * 50)

        except Exception as e:
            print(f"❌ ¡Error en el Parseo! {e}")
    else:
        print("No se encontró un resultado exitoso de ADE para probar.")
else:
    print("\nParser no inicializado o no hay resultados de ADE para probar.")



--- Parseando: 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR_text.json ---
2025-11-08 15:18:35,844 - src.extractors.structure_parser - INFO - Iniciando parseo estructurado (Markdown -> OperacionNave)...


INFO:src.extractors.structure_parser:Iniciando parseo estructurado (Markdown -> OperacionNave)...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2025-11-08 15:18:46,442 - src.extractors.structure_parser - INFO - Parseo y normalización completados.


INFO:src.extractors.structure_parser:Parseo y normalización completados.


✅ ¡Parseo Exitoso! -> 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR_structure.json
--------------------------------------------------
Recalada: None
Nave: None
Inicio de operación: None
Fin de operación: None
--------------------------------------------------
Bodegas con tonelaje: -
--------------------------------------------------
Clientes atendidos (0): -
--------------------------------------------------
Lotes ↔ Facturación: -
--------------------------------------------------
Vista inteligente:
## Operación de Nave
- **Recalada:** -
- **Nave:** -
- **Inicio:** -
- **Fin:** -

--------------------------------------------------


## 5. Resumen y Próximos Pasos

In [12]:
print("="*60)
print("RESUMEN: PASO 2 - EXTRACCIÓN Y PARSEO")
print("="*60)

print(f"✅ Se probó la API ADE ('v1/ade/parse') con {len(all_document_paths)} documentos.")
print(f"✅ La API acepta tanto PDFs como PNGs.")
print(f"✅ La API devuelve Markdown estructurado (¡mucho mejor que texto crudo!).")
print(f"✅ Se guardaron {success_count} archivos '_text.json' (con Markdown) en '{EXTRACTED_TEXT_DIR.relative_to(PROJECT_ROOT)}'.")
print(f"✅ Se probó el 'StructureParser' (Paso 3) para convertir el Markdown a JSON.")
if parser and success_count > 0:
    print(f"✅ Se guardó 1 archivo '_structure.json' en 'data/processed/extracted_tables'.")

print(f"\n🚀 PRÓXIMO PASO:")
print(f"   → Notebook 03: Generación de Embeddings (CLIP)")
print("="*60)

RESUMEN: PASO 2 - EXTRACCIÓN Y PARSEO
✅ Se probó la API ADE ('v1/ade/parse') con 9 documentos.
✅ La API acepta tanto PDFs como PNGs.
✅ La API devuelve Markdown estructurado (¡mucho mejor que texto crudo!).
✅ Se guardaron 9 archivos '_text.json' (con Markdown) en 'data\processed\extracted_text'.
✅ Se probó el 'StructureParser' (Paso 3) para convertir el Markdown a JSON.
✅ Se guardó 1 archivo '_structure.json' en 'data/processed/extracted_tables'.

🚀 PRÓXIMO PASO:
   → Notebook 03: Generación de Embeddings (CLIP)
